In [2]:
from selenium import webdriver
import time
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
from urllib.parse import urljoin
import re
import time

In [3]:
driver =  webdriver.Chrome(executable_path = 'D:\workspace\chromedriver.exe' )

driver.get('https://ev.or.kr/evmonitor')
#time.sleep(5)

In [4]:
count=0
if count==0:
    tmp_len=0
    create_date=[]

In [5]:
sid = []

In [6]:
for k in range(2,19):
    driver.find_element_by_css_selector('#F_SI_DO1')
    driver.implicitly_wait(5)
    driver.find_element_by_css_selector('#F_SI_DO1 > option:nth-child(%s)'%k).click()
    #time.sleep(3)
    driver.find_element_by_css_selector('#statList')
    driver.implicitly_wait(30)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    a = soup.find_all('div',class_='fw_path')
    for link in a:
        temp = str(link['sid'])
        sid.append(temp)


In [7]:
len(sid)

2494

In [9]:
tmp_len2 = len(sid)-tmp_len
    
if tmp_len != len(sid):
    tmp_len2 = len(sid)-tmp_len
for i in range (0,tmp_len2):
    i = time.ctime()
    create_date.append(i)
    
tmp_len = len(sid)
count+=1

In [10]:
print(len(create_date))

2494


In [11]:
name =[]  #충전소_이름
c_name=[] #충전기_충전소이름
add = []  #주소
u_time =[]  #사용가능시간
typ = []  # 구분
agency = [] # 운영기관
phone = [] # 연락처
fee = [] # 충전요금
amount = [] # 급속충전량
etc = [] # 참고사항
using = [] #사용현황
charge = [] #충전기타입
city = [] #시
gu = [] #구

In [12]:
count2=0
if count2==0:
    tmp_len_2=0
    c_create_date=[]

In [19]:
for i in sid:
    url_base = 'https://www.ev.or.kr/portal/monitor/evminfo?sid='
    url = url_base+str(i)
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    
    for tmp_using in soup.find_all(class_='table_01'):
        #if len(tmp_using)<=5:
        n_tmp = soup.find_all('h2')
        name.append(n_tmp[0].get_text())

        t_tmp = soup.find_all('em')
        u_time.append(t_tmp[0].get_text())

        info_tmp = soup.find_all(class_='table_02')
        dinfo = info_tmp[0].find_all('td')
        add.append(dinfo[0].get_text())
        agency.append(dinfo[1].get_text())
        phone.append(dinfo[2].get_text())
    
        tmp_fee = dinfo[3].get_text()
        tmp_fee = re.sub('&nbsp; | &nbsp; |\n|\t|\r|', '', tmp_fee)
        fee.append(tmp_fee)
        
            
        tmp_etc = dinfo[5].get_text()
        tmp_etc = re.sub('&nbsp; | &nbsp; |\n|\t|\r|', '', tmp_etc)
        etc.append(tmp_etc)
        
        city_tmp = dinfo[0].get_text()[0:2]
        gu_tmp = dinfo[0].get_text().split(' ')
        gu_tmp = gu_tmp[1]
        gu.append(gu_tmp)
        city.append(city_tmp)
        
        if len(tmp_using)<=5:
            
            n_tmp = soup.find_all('h2')
            c_name.append(n_tmp[0].get_text())

            ty_tmp = soup.find_all('dd')
            typ.append(ty_tmp[0].get_text())
            
            tmp_amount = dinfo[4].get_text()
            
            tmp_amount = re.findall('\d+', tmp_amount) #숫자만 추출
            amount.append(tmp_amount)
            
            for u_tmp in soup.find_all(class_='td3'):
                tmp = re.sub('\n', '', u_tmp.get_text())
                using.append(tmp)
            
    
            td2 = soup.find_all(class_='td2')
            for te in td2[0:]:
                test=[]
                for a in te.find_all('span'):
                    if a.get_text()=='사용가능':
                        break
                    elif a.get_text()=='사용중':
                        break
                    elif a.get_text()=='사용불가':
                        break
                    elif a.get_text()==' ':
                        break
                    elif len(a.get_text())<6:
                        tmp=[]
                        tmp.append(a.get_text())
                        #print(tmp)
                        test+=tmp
                test= str("+".join(test))
                charge.append(test)
                if test!=None:
                    break

                
            
        elif len(tmp_using)<=7:
            tmp_using = soup.find_all(class_='ev_type t03')
                #print(tmp_using)
            for te in tmp_using[0:]:
                    #print(te)
                for a in te.find_all('span'):
                    if a.get_text()==' ':
                        break
                    elif len(a.get_text())<6:
                        charge.append(a.get_text())

                        n_tmp = soup.find_all('h2')
                        c_name.append(n_tmp[0].get_text())

                        ty_tmp = soup.find_all('dd')
                        typ.append(ty_tmp[0].get_text())

                        tmp_amount = dinfo[4].get_text()
            
                        tmp_amount = re.findall('\d+', tmp_amount) #숫자만 추출
                        amount.append(tmp_amount)

                        for u_tmp in soup.find_all(class_='td3'):
                            tmp = re.sub('\n', '', u_tmp.get_text())
                            using.append(tmp)
                            break


URLError: <urlopen error [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다>

In [66]:
import time

In [14]:
tmp_len3 = len(c_name)-tmp_len_2
    
if tmp_len3 != len(c_name):
    tmp_len3 = len(c_name)-tmp_len_2
for i in range (0,tmp_len3):
    i = time.ctime()
    c_create_date.append(i)

tmp_len3 = len(c_name)
count2+=1

In [15]:
print(len(c_create_date), len(c_name))

14 14


In [16]:
len(name)

10

In [18]:
df = pd.DataFrame({'이름':name, '사용가능시간':u_time, '운영기관':agency,'도시':city, '구': gu,'도로명주소':add,'연락처':phone,'충전요금':fee, '참고사항':etc, '생성날짜':create_date})
df

,이름,사용가능시간,운영기관,도시,구,도로명주소,연락처,충전요금,참고사항
0,DMC 산학협력연구센터,24시간 이용가능,환경부(한국자동차환경협회),서울,마포구,서울특별시 마포구 매봉산로 37,1661-9408,유료,10분/500원
1,Park M (주차복합빌딩),24시간 이용가능,환경부(한국자동차환경협회),서울,마포구,서울특별시 마포구 매봉산로 80,1661-9408,무료,"최초30분 1500원, 10분당 500원 주민센터이용 50% 할인"
2,SETEC,24시간 이용가능,환경부(한국자동차환경협회),서울,강남구,서울특별시 강남구 남부순환로3104,1661-9408,유료,"충전차량에한하여 1시간무료 10분당500월 일주차15,000원"
3,sh서울주택도시공사,24시간 이용가능,환경부(한국자동차환경협회),서울,강남구,서울특별시 강남구 개포로 621,1661-9408,유료,없음
4,가로공원로 지하공영주차장,24시간 이용가능,환경부(한국자동차환경협회),서울,강서구,서울특별시 강서구 가로공원로 189 지하주차장 05,1661-9408,유료,주차요금 별도
5,가양레포츠센터,24시간 이용가능,환경부(한국자동차환경협회),서울,강서구,서울특별시 강서구 양천로61길 101,1661-9408,유료,없음
6,가오리 공영주차장,24시간 이용가능,환경부(한국자동차환경협회),서울,강북구,서울특별시 강북구 삼양로107길 30,1661-9408,유료,5분/150원 친환경차 50% 감면
7,강남구청 삼성로 별관 부설 주차장,09:00~18:00,환경부(한국자동차환경협회),서울,강남구,서울특별시 강남구 삼성로 628,1661-9408,무료,"5분당 300원, 주말 무료"
8,강남치매지원센터 공영노외주차장,24시간 이용가능,환경부(한국자동차환경협회),서울,강남구,서울특별시 강남구 선릉로108길 27,1661-9408,무료,5분당400원
9,강동구청,24시간 이용가능,환경부(한국자동차환경협회),서울,강동구,서울특별시 강동구 성내로 25,1661-9408,유료,5분 250원 친환경차 50% 감면


In [71]:
c_df = pd.DataFrame({'이름':c_name, '구분':typ, '충전기타입':charge, '사용중':using, '급속충전량':amount, '생성날짜':c_create_date})
c_df

,이름,구분,충전기타입,사용중,급속충전량,생성날짜
0,DMC 산학협력연구센터,급속,DC차데모+AC3상+DC콤보,사용가능,[100],Thu Sep 10 10:52:29 2020
1,Park M (주차복합빌딩),급속,DC콤보,사용가능,[200],Thu Sep 10 10:52:29 2020
2,Park M (주차복합빌딩),급속,DC콤보,사용가능,[200],Thu Sep 10 10:52:29 2020
3,SETEC,급속,DC차데모+AC3상+DC콤보,사용가능,[100],Thu Sep 10 10:52:29 2020
4,sh서울주택도시공사,급속,DC콤보,사용중,[200],Thu Sep 10 10:52:29 2020
...,...,...,...,...,...,...
2654,홍로공영주차장,급속,DC콤보,사용중,[100],Thu Sep 10 10:52:29 2020
2655,화북동주민센터,급속,DC차데모+AC3상+DC콤보,사용가능,[50],Thu Sep 10 10:52:29 2020
2656,환경성질환예방관리센터,급속,DC차데모+AC3상+DC콤보,사용가능,[50],Thu Sep 10 10:52:29 2020
2657,회수사거리 인근 공영주차장,급속,DC차데모+AC3상+DC콤보,사용가능,[50],Thu Sep 10 10:52:29 2020
